<a href="https://colab.research.google.com/github/datajoedata/e-Commerce-Sales-Analysis-Target-Brazil/blob/main/An%C3%A1lise_explorat%C3%B3ria_vendas_e_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Exploratória de E-commerce (Target)

# I - Montagem do drive e carregamento dos dados:

In [21]:
from google.colab import drive
from zipfile import ZipFile
import os
import pandas as pd
from IPython.display import display

In [22]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Descompacta arquivos

zip_path = '/content/drive/MyDrive/Analise_de_Vendas_E-commerce/archive (2).zip'

with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Lista arquivos descompactados
files = os.listdir('/content/')
print(files)


['.config', 'products.csv', 'customers.csv', 'order_items.csv', 'sellers.csv', 'payments.csv', 'geolocation.csv', 'orders.csv', 'drive', 'sample_data']


In [24]:
clientes_df = pd.read_csv('/content/customers.csv')
vendedores_df = pd.read_csv('/content/sellers.csv')
itens_pedidos_df = pd.read_csv('/content/order_items.csv')
geolocalizacao_df = pd.read_csv('/content/geolocation.csv')
pagamentos_df = pd.read_csv('/content/payments.csv')
pedidos_df = pd.read_csv('/content/orders.csv')
produtos_df = pd.read_csv('/content/products.csv')

# II -  Visualizações Iniciais e Limpeza dos dados

 Os seguintes passos foram empregados para cada DataFrame:
- **Passo 1:** Utilizar os métodos head() e tail() para visualizar as primeiras e últimas linhas.
- **Passo 2:** Verificar os tipos de dados usando df.info().
- **Passo 3:** Analisar os tipos de dados e converter conforme necessário.
- **Passo 4:** Tratar valores nulos (se existirem).
- **Passo 5:** Padronizar dados categóricos (e.g., transformar texto em minúsculas, remover espaços extras).

In [9]:
pd.set_option('display.max_rows', 5)

## Limpeza clientes

### 2.1 - Usando .head() e .tail()

head()

In [10]:
# Exibir as primeiras linhas de um DataFrame
def mostrar_head(df, nome_df):
    print(f'Primeiras linhas do DataFrame {nome_df}:')
    display(df.head())
    print('\n')

In [11]:
mostrar_head(clientes_df, 'clientes_df')

Primeiras linhas do DataFrame clientes_df:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


.tail()

In [18]:
clientes_df.tail()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS
99440,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP


### 2.2 - Usando .info

In [12]:
clientes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


### 2.3 - Remoção de duplicatas


In [13]:
def remover_duplicatas(df, coluna):
    linhas_antes = len(df)

    df_sem_duplicatas = df.drop_duplicates(subset=coluna, keep='first')

    linhas_depois = len(df_sem_duplicatas)

    linhas_removidas = linhas_antes - linhas_depois
    print(f'\nTotal de linhas removidas com base na coluna "{coluna}": {linhas_removidas}')

    return df_sem_duplicatas

In [14]:
remover_duplicatas(clientes_df, 'customer_unique_id')


Total de linhas removidas com base na coluna "customer_unique_id": 3345


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
...,...,...,...,...,...
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS
99440,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP


### 2.4 - Conversão Tipos de Dados

Converter a coluna 'customer_zip_code_prefix' para string


In [15]:
clientes_df['customer_zip_code_prefix'] = clientes_df['customer_zip_code_prefix'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB
None


Verifica alterações


In [16]:
print(clientes_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB
None


### 2.5 - Padronização

Remover espaços em branco no início e no final para todas as colunas


In [17]:
clientes_df = clientes_df.apply(lambda x: x.str.strip())

## Limpeza vendedores:

Usando .head() e .tail()

In [23]:
mostrar_head(vendedores_df, 'vendedores_df')

Primeiras linhas do DataFrame vendedores_df:


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [19]:
vendedores_df.tail()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS
3094,9e25199f6ef7e7c347120ff175652c3b,12051,taubate,SP


Usando .info

In [20]:
vendedores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


Remoção de duplicatas pela coluna id

In [26]:
vendedores_df = remover_duplicatas(vendedores_df, 'seller_id')


Total de linhas removidas com base na coluna "seller_id": 0


Conversão tipo de dados

In [ ]:
vendedores_df['seller_zip_code_prefix'] = vendedores_df['seller_zip_code_prefix'].astype(str)

Padronização

In [ ]:
# Remove espaços no começo e no final das strings
clientes_df = clientes_df.apply(lambda x: x.str.strip())

## Limpeza pedidos

In [25]:
mostrar_head(itens_pedidos_df, 'itens_pedidos_df')

Primeiras linhas do DataFrame itens_pedidos_df:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [26]:
itens_pedidos_df.tail()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72
112649,fffe41c64501cc87c801fd61db3f6244,1,350688d9dc1e75ff97be326363655e01,f7ccf836d21b2fb1de37564105216cc1,2018-06-12 17:10:13,43.00,12.79


In [ ]:
itens_pedidos_df.info()

Remoção de duplicatas

In [28]:
itens_pedidos_df = remover_duplicatas(itens_pedidos_df, 'order_id')


Total de linhas removidas com base na coluna "order_id": 13984


Conversão Tipos de dados

## Limpeza geolocalização

In [ ]:
mostrar_head(geolocalizacao_df, 'geolocalizacao_df')

Primeiras linhas do DataFrame geolocalizacao_df:


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


## Limpeza pagamentos

In [29]:
mostrar_head(pagamentos_df, 'pagamentos_df')

Primeiras linhas do DataFrame pagamentos_df:


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [30]:
pagamentos_df = remover_duplicatas(pagamentos_df, 'order_id')


Total de linhas removidas com base na coluna "order_id": 4446


## Limpeza pedidos

In [31]:
mostrar_head(pedidos_df, 'pedidos_df')

Primeiras linhas do DataFrame pedidos_df:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [32]:
pedidos_df = remover_duplicatas(pedidos_df, 'order_id')


Total de linhas removidas com base na coluna "order_id": 0


## Limpeza produtos

In [33]:
mostrar_head(produtos_df, 'produtos_df')

Primeiras linhas do DataFrame produtos_df:


,product_id,product category,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,Art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sport leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,babies,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [34]:
produtos_df = remover_duplicatas(produtos_df, 'product_id')


Total de linhas removidas com base na coluna "product_id": 0
